In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# Load and preprocess the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
#Build CNN model
model = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),  # Explicit input layer
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])


In [5]:
#Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7757 - loss: 0.6202 - val_accuracy: 0.8751 - val_loss: 0.3477
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8865 - loss: 0.3119 - val_accuracy: 0.8888 - val_loss: 0.3009
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9031 - loss: 0.2627 - val_accuracy: 0.8990 - val_loss: 0.2749
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9156 - loss: 0.2249 - val_accuracy: 0.9064 - val_loss: 0.2588
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9250 - loss: 0.1976 - val_accuracy: 0.9067 - val_loss: 0.2636


In [7]:
# Save the trained model
model.save("fashion_mnist_model.keras")

In [8]:
model.summary()
model = keras.models.load_model("fashion_mnist_model.keras")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 675,104 (2.58 MB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 450,070 (1.72 MB)

C:\Users\visha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
# Convert the model to TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply post-training quantization
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\visha\AppData\Local\Temp\tmp4ot1c1oq\assets


INFO:tensorflow:Assets written to: C:\Users\visha\AppData\Local\Temp\tmp4ot1c1oq\assets


Saved artifact at 'C:\Users\visha\AppData\Local\Temp\tmp4ot1c1oq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1661990120016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1661990112912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1661990111568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1661990112528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1661990113488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1661990125008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1661990110608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1661990125392: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [10]:
#Save the TFLite model
with open("fashion_mnist_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model training and conversion to TFLite with quantization completed.")

Model training and conversion to TFLite with quantization completed.
